In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import csv
from collections import Counter

In [2]:
sc = SparkContext()
spark = SparkSession.builder.appName('Experiment1').getOrCreate()

In [3]:
userLibraryRdd = sc.textFile("Datasets/users_libraries.txt")
userLibraryRdd = userLibraryRdd.map(lambda line: line.split(";")).map(lambda line: (line[0],list(map(int,line[1].split(",")))))

In [22]:
def processPaperCsv(line):
    paperCsv = csv.reader([line.replace("\0", "")], delimiter=',', quoting=csv.QUOTE_MINIMAL)
    paperCsvList = next(paperCsv)
    return paperCsvList[0], paperCsvList[14]

paperCsvRdd = sc.textFile("Datasets/papers.csv")
paperCsvRdd = paperCsvRdd.map(processPaperCsv).filter(lambda x: (x[1] != "" and x[1] != " ")).map(lambda x: (int(x[0]),x[1].split(" ")))

In [23]:
stopWords = sc.textFile("Datasets/stopwords_en.txt")
stopWordsBroadcast = sc.broadcast(stopWords.collect())

In [25]:
def removeStopWords(wordList):
    abstractwordsList = wordList.copy()
    for a in wordList:
        if ((a in stopWordsBroadcast.value) or a == "" or a == " "):
            abstractwordsList.remove(a)
    return abstractwordsList

userLibraryJoinPaperRdd = userLibraryRdd.flatMapValues(lambda x: x).map(lambda x: (x[1],x[0])).join(paperCsvRdd)
userLibraryJoinPaperRdd = userLibraryJoinPaperRdd.map(lambda x: (x[1][0],x[1][1]))
userLibraryJoinPaperRdd = userLibraryJoinPaperRdd.flatMapValues(lambda x:x).groupByKey().mapValues(list)
userLibraryJoinPaperWithoutStopWordsRdd = userLibraryJoinPaperRdd.mapValues(removeStopWords)

In [31]:
def findTopMostFrequentWords(x):
    CounterList = Counter(x)
    topTenMostFrequentWordWithCount = CounterList.most_common(10)
    topTenMostFrequentWord = [word for word, word_count in topTenMostFrequentWordWithCount]
    return topTenMostFrequentWord

frequentlyOccuringWordList = userLibraryJoinPaperWithoutStopWordsRdd.mapValues(findTopMostFrequentWords)

In [27]:
#frequentlyOccuringWordList.take(1)

[('2eda10fc4ff01f28b30258e7e6b7bc4b',
  ['paradigm',
   'e-learning',
   'tagclouds',
   '{ict}',
   'set',
   'effectiveness',
   'time,',
   "it's",
   'changing',
   'education'])]

In [ ]:
def CreateCsvLine(data):
    csvLineData = data[0] + "," + (','.join(str(d) for d in data[1]))
    return csvLineData

frequentlyOccuringWordListFile = frequentlyOccuringWordList.map(CreateCsvLine)
frequentlyOccuringWordListFile.saveAsTextFile("Datasets/Top10MostFrequentWordsForEachUser.csv")

In [35]:
#frequentlyOccuringWordListFile.take(4)

['06e4740007a1bda01165625971cbda70,tagging,social,tags,systems,tag,metadata,users,systems.,collaborative,web',
 '21467d75263f51fcb0e5f14e0059c158,motivation,game,intrinsic,games,research,data,learning,motivation.,gamification,software',
 '61f79c812be20b5c9e7b802702e2ab39,information,software,semantic,,knowledge,ontologies,tools,search,context,code',
 '49f5d8f90007547f86c7d38ad5ae74d3,defeasible,reasoning,nonmonotonic,rule,systems,semantic,web.,based,system,rules,']